In [1]:
import os
import librosa

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

import librosa.display

import plotly.express as px
import IPython.display as ipd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from tqdm import tqdm, trange
from librosa import feature, amplitude_to_db, load

from tqdm.auto import tqdm
from plotly.subplots import make_subplots

from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Activation , Dropout

pd.plotting.register_matplotlib_converters()

%matplotlib inline

In [4]:
import librosa
import numpy as np
import pandas as pd
import os

def extract_mfcc(audio_file, n_mfcc):
    """
    Extract MFCC features from an audio file with a frame length of 25ms
    and a hop length of 10ms for varying MFCC coefficients.
    """
    y, sr = librosa.load(audio_file, sr=None)  # Load audio with its original sampling rate
    
    hop_length = int(0.01 * sr)  # 10ms hop size
    n_fft = int(0.025 * sr)      # 25ms frame length

    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length, n_fft=n_fft)
    return mfccs.T  # Transpose to get frame-wise MFCCs

def extract_prosody_features(audio_file):
    """
    Extract prosody features like pitch and energy from an audio file.
    """
    y, sr = librosa.load(audio_file, sr=None)
    
    # Pitch (Fundamental Frequency)
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
    pitch = pitches[pitches > 0].mean() if np.any(pitches > 0) else 0  # Mean pitch
    
    # Energy (Root Mean Square Energy)
    rms = librosa.feature.rms(y=y)
    mean_energy = np.mean(rms)
    
    return {"pitch": pitch, "energy": mean_energy}

def extract_statistical_features(features):
    """
    Extract statistical features (mean, standard deviation, min, max) for given features.
    """
    stats = {
        "mean": np.mean(features, axis=0),
        "std_dev": np.std(features, axis=0),
        "min": np.min(features, axis=0),
        "max": np.max(features, axis=0)
    }
    return stats

def process_audio_folder(folder_path, n_mfcc_values):
    """
    Process all .wav audio files in a given folder to extract MFCC, prosody, and statistical features
    for multiple MFCC configurations.
    """
    audio_files = [f for f in os.listdir(folder_path) if f.endswith('.wav')]

    if not audio_files:
        print("No .wav files found in the folder!")
        return

    for n_mfcc in n_mfcc_values:
        all_features = []
        print(f"Extracting features with {n_mfcc} MFCCs...")

        for file in audio_files:
            file_path = os.path.join(folder_path, file)
            print(f"Processing: {file_path}")
            
            # Extract MFCC Features
            mfcc = extract_mfcc(file_path, n_mfcc)
            mfcc_stats = extract_statistical_features(mfcc)
            
            # Extract Prosody Features
            prosody = extract_prosody_features(file_path)
            
            # Combine Features
            combined_features = {
                "filename": file,
                **{f"MFCC_mean_{i+1}": mfcc_stats["mean"][i] for i in range(len(mfcc_stats["mean"]))},
                **{f"MFCC_std_{i+1}": mfcc_stats["std_dev"][i] for i in range(len(mfcc_stats["std_dev"]))},
                **prosody
            }
            
            all_features.append(combined_features)
        
        # Convert to DataFrame
        features_df = pd.DataFrame(all_features)

        # Save to CSV
        output_csv = f"audio_features_Dementia_{n_mfcc}_MFCC.csv"
        features_df.to_csv(output_csv, index=False)
        print(f"Features saved to {output_csv}")

# Example Usage
folder_path = r"C:\Users\adity\OneDrive\Desktop\Speech Sample\Dementia"  # Change to your folder path
n_mfcc_values = [13, 20, 30, 40, 50, 60, 70, 80]
process_audio_folder(folder_path, n_mfcc_values)


Extracting features with 13 MFCCs...
Processing: C:\Users\adity\OneDrive\Desktop\Speech Sample\Dementia\001-0.wav
Processing: C:\Users\adity\OneDrive\Desktop\Speech Sample\Dementia\001-2.wav
Processing: C:\Users\adity\OneDrive\Desktop\Speech Sample\Dementia\003-0.wav
Processing: C:\Users\adity\OneDrive\Desktop\Speech Sample\Dementia\005-0.wav
Processing: C:\Users\adity\OneDrive\Desktop\Speech Sample\Dementia\005-2.wav
Processing: C:\Users\adity\OneDrive\Desktop\Speech Sample\Dementia\007-1.wav
Processing: C:\Users\adity\OneDrive\Desktop\Speech Sample\Dementia\007-3.wav
Processing: C:\Users\adity\OneDrive\Desktop\Speech Sample\Dementia\010-0.wav
Processing: C:\Users\adity\OneDrive\Desktop\Speech Sample\Dementia\010-1.wav
Processing: C:\Users\adity\OneDrive\Desktop\Speech Sample\Dementia\010-2.wav
Processing: C:\Users\adity\OneDrive\Desktop\Speech Sample\Dementia\010-3.wav
Processing: C:\Users\adity\OneDrive\Desktop\Speech Sample\Dementia\010-4.wav
Processing: C:\Users\adity\OneDrive\Des

KeyboardInterrupt: 

In [5]:
import os
import pandas as pd

# Define the paths of both folders
folder_control = r"C:\Users\adity\OneDrive\Desktop\MFCC_NO_CONTROL"
folder_dementia = r"C:\Users\adity\OneDrive\Desktop\MFCC_NO_DIMENCIA"

# Define output folder
output_folder = r"C:\Users\adity\OneDrive\Desktop\Merged_CSV"
os.makedirs(output_folder, exist_ok=True)  # Create output folder if not exists

# Get all file names from both folders
control_files = set(f for f in os.listdir(folder_control) if f.endswith('.csv'))
dementia_files = set(f for f in os.listdir(folder_dementia) if f.endswith('.csv'))

# Find common files in both folders
common_files = control_files.intersection(dementia_files)

if not common_files:
    print("No common CSV files found in both folders.")
else:
    for file_name in common_files:
        control_path = os.path.join(folder_control, file_name)
        dementia_path = os.path.join(folder_dementia, file_name)

        # Read CSV files
        df_control = pd.read_csv(control_path)
        df_dementia = pd.read_csv(dementia_path)

        # Add a new column indicating the source
        df_control["Category"] = 0
        df_dementia["Category"] = 1

        # Merge vertically (stacking one below the other)
        merged_df = pd.concat([df_control, df_dementia], axis=0, ignore_index=True)

        # Save merged CSV file
        output_path = os.path.join(output_folder, f"Merged_{file_name}")
        merged_df.to_csv(output_path, index=False)

        print(f"Merged file saved: {output_path}")


Merged file saved: C:\Users\adity\OneDrive\Desktop\Merged_CSV\Merged_audio_features_60_MFCC.csv
Merged file saved: C:\Users\adity\OneDrive\Desktop\Merged_CSV\Merged_audio_features_20_MFCC.csv
Merged file saved: C:\Users\adity\OneDrive\Desktop\Merged_CSV\Merged_audio_features_50_MFCC.csv
Merged file saved: C:\Users\adity\OneDrive\Desktop\Merged_CSV\Merged_audio_features_13_MFCC.csv
Merged file saved: C:\Users\adity\OneDrive\Desktop\Merged_CSV\Merged_audio_features_80_MFCC.csv
Merged file saved: C:\Users\adity\OneDrive\Desktop\Merged_CSV\Merged_audio_features_30_MFCC.csv
Merged file saved: C:\Users\adity\OneDrive\Desktop\Merged_CSV\Merged_audio_features_40_MFCC.csv
Merged file saved: C:\Users\adity\OneDrive\Desktop\Merged_CSV\Merged_audio_features_70_MFCC.csv


In [7]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Define folder path
folder_path = r"C:\Users\adity\OneDrive\Desktop\MFCC_NO_MERGED"

# Read all CSV files and merge them
all_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

if not all_files:
    print("No CSV files found in the folder!")
    exit()

# List to store dataframes
dataframes = []

for file_name in all_files:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    dataframes.append(df)

# Merge all data
final_df = pd.concat(dataframes, ignore_index=True)

# Check dataset
print("Dataset Shape:", final_df.shape)
print(final_df.head())

# Drop any unnecessary columns (filename, etc.)
if 'filename' in final_df.columns:
    final_df.drop(columns=['filename'], inplace=True)

# Handle missing values
final_df.dropna(inplace=True)

# Encode categorical labels if necessary
if 'Category' in final_df.columns:  # Assuming "Category" has 'Control' & 'Dementia'
    label_encoder = LabelEncoder()
    final_df['Category'] = label_encoder.fit_transform(final_df['Category'])  # Converts to 0,1

# Split dataset into features (X) and labels (y)
X = final_df.drop(columns=['Category'])  # All columns except the target
y = final_df['Category']  # Target column

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into training & testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Train SVM model
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train, y_train)

# Predictions
y_pred = svm_model.predict(X_test)

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
print("\nModel Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_test, y_pred))




Dataset Shape: (2040, 164)
    filename  MFCC_mean_1  MFCC_mean_2  MFCC_mean_3  MFCC_mean_4  MFCC_mean_5  \
0  002-0.wav   -511.20593     85.60844    -3.060893    28.423433   -16.799236   
1  002-1.wav   -484.06348    109.80636    -4.809857    31.444185    -5.454689   
2  002-2.wav   -660.31366     95.91071    17.194544    36.756440   -11.440260   
3  002-3.wav   -438.16710    130.54025    -4.966709    19.657808    -4.148704   
4  006-2.wav   -500.21326    126.47529    -0.818541    26.768965     5.693782   

   MFCC_mean_6  MFCC_mean_7  MFCC_mean_8  MFCC_mean_9  ...  MFCC_std_71  \
0    32.041927   -15.167766    10.931705    -6.686311  ...          NaN   
1    14.465662    -9.392167    16.793055   -11.293805  ...          NaN   
2    27.625645   -15.950785    21.202023   -14.688868  ...          NaN   
3     5.476681   -15.456863     1.497129   -12.413127  ...          NaN   
4    -2.505827     7.964436    -1.402581   -15.316589  ...          NaN   

   MFCC_std_72  MFCC_std_73  MFCC_s

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Folder containing merged CSV files
input_folder = r"C:\Users\adity\OneDrive\Desktop\MFCC_NO_MERGED"

# List to store model performance results
results = []

# Get all CSV files in the folder
csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

if not csv_files:
    print("No CSV files found in the folder.")
else:
    for file_name in csv_files:
        file_path = os.path.join(input_folder, file_name)
        print(f"Processing: {file_name}")

        # Load dataset
        df = pd.read_csv(file_path)

        # Ensure there is a label column (assumes a "Category" column exists)
        if "Category" not in df.columns:
            print(f"Skipping {file_name} (No 'Category' column found)")
            continue
        
        # Drop non-numeric columns (e.g., filename) from the features
        # Keep only numeric columns and exclude the label
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        if "Category" not in numeric_cols:
            print(f"Skipping {file_name} (Label column not numeric)")
            continue

        X = df[numeric_cols].drop(columns=["Category","energy","pitch"])
        y = df["Category"]

        # Standardize features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # Split into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

        # Train SVM model
        model = SVC(kernel='linear', random_state=42)
        model.fit(X_train, y_train)

        # Make predictions and compute accuracy
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        # Store results
        results.append({"Filename": file_name, "Accuracy": accuracy})
        print(f"Model trained for {file_name} | Accuracy: {accuracy:.4f}")

# Convert results to DataFrame and save to CSV
results_df = pd.DataFrame(results)
output_csv = os.path.join(input_folder, "SVM_Model_Results.csv")
results_df.to_csv(output_csv, index=False)

print(f"\nSVM Model Results saved to: {output_csv}")


Processing: Merged_audio_features_13_MFCC.csv
Model trained for Merged_audio_features_13_MFCC.csv | Accuracy: 0.6667
Processing: Merged_audio_features_20_MFCC.csv
Model trained for Merged_audio_features_20_MFCC.csv | Accuracy: 0.7255
Processing: Merged_audio_features_30_MFCC.csv
Model trained for Merged_audio_features_30_MFCC.csv | Accuracy: 0.6471
Processing: Merged_audio_features_40_MFCC.csv
Model trained for Merged_audio_features_40_MFCC.csv | Accuracy: 0.5294
Processing: Merged_audio_features_50_MFCC.csv
Model trained for Merged_audio_features_50_MFCC.csv | Accuracy: 0.6471
Processing: Merged_audio_features_60_MFCC.csv
Model trained for Merged_audio_features_60_MFCC.csv | Accuracy: 0.6471
Processing: Merged_audio_features_70_MFCC.csv
Model trained for Merged_audio_features_70_MFCC.csv | Accuracy: 0.5882
Processing: Merged_audio_features_80_MFCC.csv
Model trained for Merged_audio_features_80_MFCC.csv | Accuracy: 0.6275
Processing: SVM_Model_Results.csv
Skipping SVM_Model_Results.csv